In [60]:
import pandas as pd
import numpy as np

In [61]:
import edhec_risk_kit_204 as erk

# Load data

In [62]:
# industries_to_analysis = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food',]

In [63]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2014":"2018"]
# ind49_rets = ind49_rets[industries_to_analysis]
cov = ind49_rets.cov()

In [64]:
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2014":"2018"]
# ind_mcap = ind_mcap[industries_to_analysis]

In [65]:
cap_weigth = (ind_mcap.iloc[0] / ind_mcap.iloc[0].sum())
cap_weigth.sort_values().head()

FabPr    0.000271
Gold     0.000686
Ships    0.000860
Coal     0.000893
Txtls    0.000970
Name: 2014-01, dtype: float64

In [66]:
def risk_contribution(w,cov):
    """
    Compute the contributions to risk of the constituents of a portfolio, given a set of portfolio weights and a covariance matrix
    """
    total_portfolio_var = erk.portfolio_vol(w,cov)**2
    # Marginal contribution of each constituent
    marginal_contrib = cov@w
    risk_contrib = np.multiply(marginal_contrib,w.T)/total_portfolio_var
    return risk_contrib

In [67]:
from scipy.optimize import minimize

def target_risk_contributions(target_risk, cov):
    """
    Returns the weights of the portfolio that gives you the weights such
    that the contributions to portfolio risk are as close as possible to
    the target_risk, given the covariance matrix
    """
    n = cov.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),) * n # an N-tuple of 2-tuples!
    # construct the constraints
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    def msd_risk(weights, target_risk, cov):
        """
        Returns the Mean Squared Difference in risk contributions
        between weights and target_risk
        """
        w_contribs = risk_contribution(weights, cov)
        return ((w_contribs-target_risk)**2).sum()
    
    weights = minimize(msd_risk, init_guess,
                       args=(target_risk, cov), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,),
                       bounds=bounds)
    return weights.x

def equal_risk_contributions(cov):
    """
    Returns the weights of the portfolio that equalizes the contributions
    of the constituents based on the given covariance matrix
    """
    n = cov.shape[0]
    return target_risk_contributions(target_risk=np.repeat(1/n,n), cov=cov)


# Assignment

In [68]:
risk_contribution(cap_weigth, cov).sort_values().to_frame("Cap weighted").iloc[[0,1,2,-3,-2,-1]].style.format("{:.2%}")

,Cap weighted
Gold,0.01%
FabPr,0.03%
Coal,0.09%
Oil,8.89%
Softw,9.37%
Banks,10.41%


In [69]:
f"{np.ptp(risk_contribution(cap_weigth, cov)):.2%}"

'10.40%'

In [70]:
risk_contribution(erk.weight_ew(ind49_rets), cov).sort_values().to_frame("Equal weighted").iloc[[0,1,2,-3,-2,-1]].style.format("{:.2%}")

,Equal weighted
Util,0.59%
Smoke,0.94%
Hshld,1.05%
Coal,2.90%
Ships,3.00%
Steel,3.09%


In [71]:
f"{np.ptp(risk_contribution(erk.weight_ew(ind49_rets), cov)):.2%}"

'2.50%'

In [72]:
pd.Series(equal_risk_contributions(cov), index=cov.index).sort_values().to_frame("Equal Risk - Weights").iloc[[0,1,2,-3,-2,-1]].style.format("{:.2%}")

,Equal Risk - Weights
Steel,1.28%
Ships,1.30%
BldMt,1.38%
Smoke,3.26%
Hshld,3.32%
Util,5.22%


In [73]:
risk_contribution(equal_risk_contributions(cov), cov).sort_values().to_frame("Equal Risk - Risk Contributions").iloc[[0,1,2,-3,-2,-1]].style.format("{:.2%}")

,Equal Risk - Risk Contributions
Util,2.02%
Steel,2.03%
Ships,2.03%
Other,2.05%
Beer,2.05%
Hshld,2.05%
